In [1]:
pip install trtokenizer

In [12]:
import pandas as pd
import re
import string
from trtokenizer.tr_tokenizer import SentenceTokenizer
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('stopwords')
from snowballstemmer import stemmer
from gensim.models import FastText
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import spacy
from spacy.lang.tr.stop_words import STOP_WORDS



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
df = pd.read_excel('tweetData.xlsx')

In [4]:
df.shape

(15604, 2)

In [5]:
df = df.dropna()

In [6]:
df.shape

(1976, 2)

In [7]:
def remove_special_characters(text):
    try:
      text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
      return text
    except:
        pass

def to_lowercase(text):
    try:
      return text.lower()
    except:
        pass

def remove_extra_spaces(text):
    try:
      return  ' '.join(text.split())
    except:
        pass

def remove_html_tags(text):
    try:
      clean = re.compile('<.*?>')
      return re.sub(clean, '', text)
    except:
        pass

def remove_urls(text):
    try:
      return re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    except:
        pass

def remove_numbers(text):
    try:
      return re.sub(r'\d+', '', text)
    except:
        pass

def remove_punctuation(text):
    try:
      translator = str.maketrans('', '', string.punctuation)
      return text.translate(translator)
    except:
      pass

In [8]:
df['Tweets'] = df['Tweets'].apply(to_lowercase).apply(remove_extra_spaces).apply(remove_punctuation).apply(remove_html_tags).apply(remove_urls).apply(remove_numbers)

<ipython-input-8-acdcc727262d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Tweets'] = df['Tweets'].apply(to_lowercase).apply(remove_extra_spaces).apply(remove_punctuation).apply(remove_html_tags).apply(remove_urls).apply(remove_numbers)


In [9]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df.Tweets.loc[15595:15599]

15595    kimmiş lan rahatsız olacak söylesin infaz edel...
15596                              i̇nstasina infaz edelim
15597    üstad latin abd ülkelerinde denediler direk ka...
15598    bu iki oevladının açık adreslerini verin infaz...
15599     kıbrısın tamamı alınmalı   ada diye bize yutt...
Name: Tweets, dtype: object

In [10]:
df.head()

,Tweets,Etiket
0,aynı bayrak aynı göğün altındayız biriz berabe...,1.0
1,çok istekli olan bu erkekler lütfen erkek çoc...,1.0
2,kurtlerle komşumuz alrabayiz beraberiz siz yal...,2.0
3,müslümanın seni beni ırkı olmaz dini olur kims...,1.0
4,soramazsına hoşgeldiniz bu gün bilge ile berab...,2.0


In [26]:


# Cümle ve kelime ayrıştırıcı işlevler
def trtokenizer_sentence(text):
    try:
        return sent_tokenize(text, language='turkish')
    except Exception as e:
        print(f"Sentence tokenization error: {e}")
        return []

def trtokenizer_word(text):
    try:
        return word_tokenize(text, language='turkish')
    except Exception as e:
        print(f"Word tokenization error: {e}")
        return []

# Stop kelimeleri temizleme işlevi
def remove_stopwords_nltk(words):
    stop_words = set(stopwords.words('turkish'))
    return [word for word in words if word.lower() not in stop_words]

def snowballstemmer(text):

  # Snowball Stemmer oluşturulması
  turkish_stemmer = stemmer("turkish")
  # Stem yapılması
  stemmed_words = [turkish_stemmer.stemWord(word) for word in text]
  return ' '.join(stemmed_words)




# Cümle ve kelime ayrıştırma işlemlerini DataFrame'e uygulama
df['sentences'] = df['Tweets'].apply(trtokenizer_sentence)

# Kelimeleri çıkarmak için her cümleyi kelimelere böler ve tüm kelimeleri düz bir liste olarak toplar
df['words'] = df['sentences'].apply(lambda sentences: [word for sentence in sentences for word in trtokenizer_word(sentence)])

# Stop kelimeleri temizleme işlemini ve birleştirme işlemini DataFrame'e uygulama
df['cleaned_words'] = df['words'].apply(remove_stopwords_nltk)
df['lemmatized_words'] = df['cleaned_words'].apply(snowballstemmer)

# Sonuçları yazdırma
print(df.head())




                                              Tweets  Etiket  \
0  aynı bayrak aynı göğün altındayız biriz berabe...     1.0   
1  çok istekli olan bu erkekler  lütfen erkek çoc...     1.0   
2  kurtlerle komşumuz alrabayiz beraberiz siz yal...     2.0   
3  müslümanın seni beni ırkı olmaz dini olur kims...     1.0   
4  soramazsına hoşgeldiniz bu gün bilge ile berab...     2.0   

                                           sentences  \
0  [aynı bayrak aynı göğün altındayız biriz berab...   
1  [çok istekli olan bu erkekler  lütfen erkek ço...   
2  [kurtlerle komşumuz alrabayiz beraberiz siz ya...   
3  [müslümanın seni beni ırkı olmaz dini olur kim...   
4  [soramazsına hoşgeldiniz bu gün bilge ile bera...   

                                               words  \
0  [aynı, bayrak, aynı, göğün, altındayız, biriz,...   
1  [çok, istekli, olan, bu, erkekler, lütfen, erk...   
2  [kurtlerle, komşumuz, alrabayiz, beraberiz, si...   
3  [müslümanın, seni, beni, ırkı, olmaz, dini, ol...  

In [25]:

# Kelimeleri tokenize et
tokenize_metni = [metin.lower().split() for metin in df['lemmatized_words']]

# FastText modelini eğit
fasttext_model = FastText(vector_size=100, window=3, min_count=1, epochs=10)
fasttext_model.build_vocab(corpus_iterable=tokenize_metni)
fasttext_model.train(corpus_iterable=tokenize_metni, total_examples=len(tokenize_metni), epochs=10)

# Her metin için ortalama kelime vektörlerini hesapla
def ortalama_vektor(metin):
    kelimeler = metin.lower().split()
    vektörler = [fasttext_model.wv[kelime] for kelime in kelimeler if kelime in fasttext_model.wv]
    if vektörler:
        return np.mean(vektörler, axis=0)
    else:
        return np.zeros(100)

df['fasttext_vektor'] = df['lemmatized_words'].apply(ortalama_vektor)
print(df['fasttext_vektor'] )



0        [-0.49059707, -0.120382115, -0.23167974, 0.208...
1        [-0.563759, -0.13745905, -0.26632476, 0.239139...
2        [-0.34694582, -0.08502632, -0.16296573, 0.1470...
3        [-0.5623434, -0.13765398, -0.2662421, 0.239221...
4        [-0.46170115, -0.113285325, -0.21778186, 0.196...
5        [-0.39239797, -0.09617918, -0.18488337, 0.1655...
6        [-0.47901928, -0.11709208, -0.22635642, 0.2033...
7        [-0.5951902, -0.14582485, -0.28099766, 0.25253...
8        [-0.57636875, -0.14030021, -0.272491, 0.244310...
9        [-0.57915395, -0.14142491, -0.2736609, 0.24659...
10       [-0.41786128, -0.10242601, -0.1970157, 0.17766...
11       [-0.47051153, -0.115478456, -0.22294016, 0.199...
12       [-0.57122725, -0.1394987, -0.27059823, 0.24345...
13       [-0.4815278, -0.11739378, -0.22781396, 0.20411...
14       [-0.5291799, -0.12955445, -0.25009766, 0.22428...
15       [-0.42644784, -0.10430652, -0.2016121, 0.18093...
16       [-0.5802797, -0.14224628, -0.2748441, 0.246264.